In [13]:
import os
import json
import uuid

import utils_elasticsearch as utils

from elasticsearch import helpers

In [8]:
es = utils.connect_elasticsearch()

Connected to <Elasticsearch([{'host': 'localhost', 'port': 9200}])>


In [4]:
def read_json_file(filename):

    with open(filename, 'r') as f:
        lines = f.readlines()

    json_data = []
    for line in lines:
        temp_json = json.loads(line)['data']
        
        json_data.append({
            'name'    : temp_json['name'],
            'state'   : temp_json['state'],
            'pledged' : temp_json['pledged'],
            'creator' : temp_json['creator']['name']
        })

    return json_data

json_data = read_json_file('../data/kickstarter_test.json')

In [9]:
actions = [
    {
        "_id"      : uuid.uuid4(), # random UUID for _id
        "doc_type" : "_doc", # document _type
        "doc"      : doc,
        
    }
    for doc in json_data
]

In [11]:
json_data

[{'name': 'Palmetto Pork Skins',
  'state': 'failed',
  'pledged': 50,
  'creator': 'Palmetto Pork Skins (deleted)'},
 {'name': 'Photography Gloves: Extend your session in style',
  'state': 'successful',
  'pledged': 355520,
  'creator': 'Vallerret Photography Gloves'},
 {'name': 'Eat Shit: A Memorial Pin Campaign',
  'state': 'successful',
  'pledged': 253,
  'creator': 'Enzo Daniel'},
 {'name': 'TRIC Clothing Co.',
  'state': 'successful',
  'pledged': 4475,
  'creator': 'Brandon Maddux'},
 {'name': "Women's Gold Medalists Rio 2016",
  'state': 'successful',
  'pledged': 50899,
  'creator': 'Wendy Fox'}]

In [14]:
response = helpers.bulk(es, actions, index='forward43')

In [15]:
response

(5, [])

In [16]:
def bulk_json_data():

    for doc in json_data:
        if '{"index"' not in doc:
            yield {
                "_index"  : 'forward43',
                "_type"   : '_doc',
                "_id"     : uuid.uuid4(),
                "_source" : doc
            }

In [17]:
response = helpers.bulk(es, bulk_json_data())

/home/stroopwafel/miniconda3/envs/whispp_env/lib/python3.7/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchWarning)


In [18]:
response

(5, [])